In [1]:
!git clone https://github.com/NVIDIA/apex

Cloning into 'apex'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 5855 (delta 10), reused 9 (delta 7), pack-reused 5835
Receiving objects: 100% (5855/5855), 13.59 MiB | 14.13 MiB/s, done.
Resolving deltas: 100% (3802/3802), done.


In [19]:
!cd apex && pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

/opt/anaconda3/lib/python3.7/site-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-s30eqda5
Created temporary directory: /tmp/pip-req-tracker-f5xop5nj
Created requirements tracker '/tmp/pip-req-tracker-f5xop5nj'
Created temporary directory: /tmp/pip-install-ef575ix1
Processing /home/jupyter/apex
  Created temporary directory: /tmp/pip-req-build-caz0q0z5
  Added file:///home/jupyter/apex to build tracker '/tmp/pip-req-tracker-f5xop5nj'
    Running setup.py (path:/tmp/pip-req-build-caz0q0z5/setup.py) egg_info for package from file:///home/jupyter/apex
    Running command python setup.py egg_info
    torch.__version__  =  1.3.1
    running egg_info
    creating /tmp/pip-req-build-caz0q0z5/pip-egg-info/apex.egg-info
    writing /tmp/pip-req-build-caz0q0z5/pip-egg-info/apex.e

In [1]:
import numpy as np
import torch
import copy
import torch.nn.functional as F
from torch.nn import DataParallel

from torch import nn, optim
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
multi_gpus = False
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.device_count() > 1:
    multi_gpus = True
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
class ConvBlock(nn.Module):

    def __init__(self, in_planes, out_planes, kernel_size, stride=1, padding=True):

        super(ConvBlock, self).__init__()
        self.kernel_size = kernel_size
        self.stride = stride
        pad = 0
        if padding :
            pad = (self.kernel_size - 1) // 2

        self.conv = nn.Conv2d(in_planes, out_planes, kernel_size, stride, padding=pad, bias=False)
        self.batchnorm = nn.BatchNorm2d(out_planes, momentum=0.99)
        self.leaky_relu = nn.LeakyReLU(0.1, inplace=True)

    def forward(self, x):

        output = self.conv(x)
        output = self.batchnorm(output)
        output = self.leaky_relu(output)

        return output

In [4]:
class DarknetBlock(nn.Module):

    def __init__(self, out_planes):

        super(DarknetBlock, self).__init__()
        self.inplanes = out_planes * 2
        self.conv1 = ConvBlock(self.inplanes, out_planes, 1)
        self.conv2 = ConvBlock(out_planes, self.inplanes, 3)
        #self.se = SEBlock(self.inplanes, ratio=9)

    def forward(self, x):

        shortcut = x
        output = self.conv1(x)
        output = self.conv2(output)
        #output = self.se(output)
        output = output + shortcut

        return output

In [5]:
class Darknet(nn.Module):

    def __init__(self, num_classes=1000):

        super(Darknet, self).__init__()   

        self.conv_block1 = ConvBlock(3, 32, 3, 1)
        self.conv_block2 = ConvBlock(32, 64, 3, 2)

        self.dark_block1 = DarknetBlock(32)

        self.conv_block3 = ConvBlock(64, 128, 3, 2)

        self.dark_layer1 = self._make_blocks(2, 64)
        
        self.conv_block4 = ConvBlock(128, 256, 3, 2)

        self.dark_layer2 = self._make_blocks(8, 128)

        self.conv_block5 = ConvBlock(256, 512, 3, 2)

        self.dark_layer3 = self._make_blocks(8, 256)

        self.conv_block6 = ConvBlock(512, 1024, 3, 2)

        self.dark_layer4 = self._make_blocks(4, 512)

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        
        self.fc = nn.Linear(1024, num_classes)

    def _make_blocks(self, num_blocks, out_planes):
        blocks = []
        for _ in range(num_blocks):
            blocks.append(DarknetBlock(out_planes))

        return nn.Sequential(*blocks)

    def forward(self, x, feature=False):

        output = self.conv_block1(x)
        output = self.conv_block2(output)

        output = self.dark_block1(output)

        output = self.conv_block3(output)

        output = self.dark_layer1(output)

        output = self.conv_block4(output)

        output = self.dark_layer2(output)

        output = self.conv_block5(output)

        output = self.dark_layer3(output)

        output = self.conv_block6(output)

        output = self.dark_layer4(output)

        output = self.avgpool(output)

        output = torch.flatten(output, 1)
        
        if feature:
            return output

        output = self.fc(output)
        
        return output

In [6]:
model = Darknet(num_classes=150)

In [7]:
inputs = torch.randn((1,3,224,224))
outputs = model(inputs)

In [8]:
outputs.shape

torch.Size([1, 150])

In [9]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [10]:
count_parameters(model)

40738678

In [11]:
import apex
print("using apex synced BN")
model = apex.parallel.convert_syncbn_model(model)

using apex synced BN


In [12]:
optimizer = optim.SGD(model.parameters(), lr=1., momentum=0.9, weight_decay=1e-4, nesterov=True)

In [13]:
from apex import amp, optimizers

model, optimizer = amp.initialize(model.cuda(), optimizer, opt_level='O3',keep_batchnorm_fp32=True)

Selected optimization level O3:  Pure FP16 training.
Defaults for this optimization level are:
enabled                : True
opt_level              : O3
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : False
master_weights         : False
loss_scale             : 1.0
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O3
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : True
master_weights         : False
loss_scale             : 1.0


In [14]:
import torchvision
from torchvision import datasets, transforms

def get_transform(random_crop=True):
    normalize = transforms.Normalize(
        mean=[x / 255.0 for x in [125.3, 123.0, 113.9]],
        std=[x / 255.0 for x in [63.0, 62.1, 66.7]])
    transform = []
    transform.append(transforms.Resize(256))
    if random_crop:
        transform.append(transforms.RandomResizedCrop(224))
        transform.append(transforms.RandomHorizontalFlip())
    else:
        transform.append(transforms.CenterCrop(224))
    transform.append(transforms.ToTensor())
    transform.append(normalize)
    return transforms.Compose(transform)

class CustomDataset(datasets.ImageFolder):
    def __getitem__(self, index):
        """
        Args:
            index (int): Index

        Returns:
            tuple: (image_id, sample, target) where target is class_index of
                the target class.
        """
        path, target = self.samples[index]
        #print(path)
        #print(target)
        sample = self.loader(path)
        if self.transform is not None:
            sample = self.transform(sample)
        if self.target_transform is not None:
            target = self.target_transform(target)

        image_id = path.split('/')[-1]

        return image_id, sample, target

In [15]:
from torch.utils import data

data_dir = 'train/train_data'

dataset = CustomDataset(data_dir, transform=get_transform(random_crop=True))

split_size = int(len(dataset) * 0.9)
train_set, valid_set = data.random_split(dataset, [split_size, len(dataset) - split_size])
tr_loader = data.DataLoader(dataset=train_set,
                            batch_size=256,
                            #sampler = RandomIdentitySampler(train_set, 4),
                            shuffle=True,
                            pin_memory=True,
                            num_workers=16)

val_loader = data.DataLoader(dataset=valid_set,
                             batch_size=256,
                             shuffle=False,
                            pin_memory=True,                             
                            num_workers=16)

In [16]:
import warnings
warnings.filterwarnings('ignore')

In [17]:
criterion = nn.CrossEntropyLoss().cuda()

In [18]:
torch.cuda.synchronize()
model.train()
for _ in range(2):
    _, inputs, labels = next(iter(tr_loader))
    print(1)
    inputs = inputs.cuda(non_blocking=True)        
    labels = labels.cuda(non_blocking=True)    
    print(2)    
    logits = model(inputs)
    print(3)                       
    loss = criterion(logits, labels)                   
    print(4)                   
    with amp.scale_loss(loss, optimizer) as scaled_loss:
        scaled_loss.backward()
    print(5)                            
    model.zero_grad()
    print(10)                                
torch.cuda.synchronize()

1
2
3
4
5
10
1
2
3
4
5
10


In [19]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [20]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [21]:
len(tr_loader)

365

In [22]:
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.1, steps_per_epoch=len(tr_loader)
                                                , epochs=30, pct_start=0.2)

In [23]:
import datetime
import time
high = 0.0
epoch_time = AverageMeter('Epoch', ':6.3f')
batch_time = AverageMeter('Batch', ':6.3f')
data_time = AverageMeter('Data', ':6.3f')
losses = AverageMeter('Loss', ':.5f')
learning_rates = AverageMeter('LearningRate', ':.5f')
top1 = AverageMeter('Acc@1', ':6.2f')
top5 = AverageMeter('Acc@5', ':6.2f')

for epoch in range(30):  # loop over the dataset multiple times
    time_ = datetime.datetime.now()    
    model.train()
    running_loss = 0.0
    running_corrects = 0
    total = 0
    progress = ProgressMeter(
        len(tr_loader),
        [batch_time, data_time, losses, top1, top5, learning_rates],
        prefix="Epoch: [{}]".format(epoch))
    
    end = time.time()    
    for i, (_, inputs, labels) in enumerate(tr_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        #print(inputs.shape)
        #print(labels.shape)
        data_time.update(time.time() - end)
        inputs = inputs.cuda(non_blocking=True)
        labels = labels.cuda(non_blocking=True)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        #_, preds = torch.max(outputs, 1)
        #loss.backward()
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
            
        optimizer.step()
        scheduler.step()
        # print statistics
        acc1, acc5 = accuracy(outputs, labels, topk=(1, 5))
        losses.update(loss.item(), inputs.size(0))
        learning_rates.update(scheduler.get_lr()[0])        
        top1.update(acc1[0], inputs.size(0))
        top5.update(acc5[0], inputs.size(0))

        
        batch_time.update(time.time() - end)
        if i % 100 == 99:    # print every 2000 mini-batches
            progress.display(i)
            #running_loss = 0.0
    elapsed = datetime.datetime.now() - time_
    print('{} elapsed for {}'.format(elapsed, epoch+1))

    
print('Finished Training')

Epoch: [0][ 99/365]	Batch 206.132 (107.548)	Data 205.739 (107.155)	Loss 4.26110 (4.47482)	Acc@1   6.25 (  5.14)	Acc@5  24.61 ( 17.48)	LearningRate 0.00449 (0.00417)
Epoch: [0][199/365]	Batch 372.612 (197.851)	Data 372.219 (197.457)	Loss 3.77385 (4.19395)	Acc@1  11.33 (  8.31)	Acc@5  41.80 ( 25.32)	LearningRate 0.00596 (0.00466)
Epoch: [0][299/365]	Batch 553.796 (287.754)	Data 553.403 (287.361)	Loss 3.46083 (3.99774)	Acc@1  16.80 ( 10.93)	Acc@5  45.70 ( 30.88)	LearningRate 0.00838 (0.00548)
0:10:40.517552 elapsed for 1
Epoch: [1][ 99/365]	Batch 62.322 (278.197)	Data 61.928 (277.801)	Loss 3.02011 (3.75576)	Acc@1  30.86 ( 14.49)	Acc@5  55.86 ( 37.80)	LearningRate 0.01430 (0.00750)
Epoch: [1][199/365]	Batch 108.511 (244.117)	Data 108.107 (243.718)	Loss 2.99504 (3.63401)	Acc@1  22.66 ( 16.47)	Acc@5  60.94 ( 41.16)	LearningRate 0.01893 (0.00910)
Epoch: [1][299/365]	Batch 158.867 (227.653)	Data 158.475 (227.252)	Loss 2.96040 (3.52655)	Acc@1  30.86 ( 18.29)	Acc@5  60.94 ( 44.03)	LearningRate 0

In [24]:
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'loss': loss,    
    
}, './checkpoint/darknet53_fp16_sconv_ep030.b0.pth')

In [25]:
def classification_val(model, val_loader):
    correct = 0
    total = 0    
    
    model.eval()
    with torch.no_grad():
        for data in val_loader:
            _, images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct/total

In [26]:
cls_results = [classification_val(model, val_loader) for i in range(10)]

In [27]:
np.mean(cls_results)

0.7655906725766044

In [28]:
np.std(cls_results)

0.002814254666530955

In [29]:
cls_results

[0.7619965311235305,
 0.7655617652726923,
 0.7658508383118134,
 0.766525342069763,
 0.7618038157641164,
 0.7669107727885912,
 0.7698015031798034,
 0.7661399113509346,
 0.7617074580844093,
 0.7696087878203893]

In [30]:
def val_retrieval(model, val_loader):
    feats = None
    data_ids = None

    model.eval()
    with torch.no_grad():
        for idx, (_, images, labels) in enumerate(val_loader):
            images = images.to(device)
            #labels = labels.to(device)

            feat = model(images, feature=True)
            feat = feat.detach().cpu().numpy()

            feat = feat/np.linalg.norm(feat, axis=1)[:, np.newaxis]

            if feats is None:
                feats = feat
            else:
                feats = np.append(feats, feat, axis=0)

            if data_ids is None:
                data_ids = labels
            else:
                data_ids = np.append(data_ids, labels, axis=0)

        score_matrix = feats.dot(feats.T)
        np.fill_diagonal(score_matrix, -np.inf)
        top1_reference_indices = np.argmax(score_matrix, axis=1)

        top1_reference_ids = [
            [data_ids[idx], data_ids[top1_reference_indices[idx]]] for idx in
            range(len(data_ids))]

    total_count = len(top1_reference_ids)
    correct = 0
    for ids in top1_reference_ids:
        if ids[0] == ids[1]:
            correct += 1        
    return correct/total_count

In [31]:
retrieval_result = [val_retrieval(model, val_loader) for i in range(10)]

In [33]:
np.mean(retrieval_result)

0.6285122374253228

In [34]:
np.std(retrieval_result)

0.0027957180724779034

In [35]:
retrieval_result

[0.6331663133551744,
 0.628637502408942,
 0.6262285604162652,
 0.625939487377144,
 0.6291192908074773,
 0.631239159761033,
 0.6304682983233764,
 0.6230487569859318,
 0.6298901522451339,
 0.62738485257275]